In [1]:
import sys
sys.path.insert(0,'/home/caldaz/module/pyGSM')
from molecule import Molecule
from pes import PES
from avg_pes import Avg_PES
import numpy as np
from nifty import pvec1d,pmat2d
import matplotlib
import matplotlib.pyplot as plt
from pytc import *
import manage_xyz
from rhf_lot import *
from psiw import *
from nifty import getAllCoords,getAtomicSymbols,click,printcool
import pybel as pb
%matplotlib inline

In [2]:
printcool("Build resources")
resources = ls.ResourceList.build()
printcool('{}'.format(resources))

#========================================================#
#|                   Build resources                    |#
#========================================================#
#==============================================================#
#|                       ResourceList:                        |#
#|                       CPU threads:   4                     |#
#|                       GPU contexts:  1                     |#
#|                     GPU context details:                   |#
#|    N  ID                 Name  CC  CPU Buffer  GPU Buffer  |#
#|    0   0    GeForce GTX TITAN 3.5        1024        1024  |#
#|                                                            |#
#==============================================================#


'----------------------------------------------------------------\n'

In [3]:
printcool("build the Lightspeed (pyTC) objecs")
qmmm = QMMM.from_prmtop(
    prmtopfile='data/system.prmtop',
    inpcrdfile='data/system.inpcrd',
    qmindsfile='data/system.qm',
    charge=0.,
    )
geom = geometry.Geometry.build(
    resources=resources,
    qmmm=qmmm,
    basisname='3-21g',
    )
print(geom)
ref = RHF(RHF.default_options().set_values({
    'geometry' : geom, 
    'dft_functional' : 'B3LYP',
    'dft_grid_name' : 'SG0',
    }))
psiw = RHF_LOT.from_options(rhf=ref)


#========================================================#
#|          build the Lightspeed (pyTC) objecs          |#
#========================================================#
Geometry:
  QMMM = True
  -D3  = False
  ECP  = False

QM/MM Details:
  Total atoms   = 2774
  QM atoms      = 56
  Link H atoms  = 0

Molecule: QM/MM Full Molecule
  Natom        =        2774
  Charge       =       0.000
  Multiplicity =       1.000

Molecule: QM/MM QM Molecule
  Natom        =          56
  Charge       =       0.000
  Multiplicity =       1.000

Basis: 3-21g
  nao     =   308
  ncart   =   308
  nprim   =   336
  nshell  =   196
  natom   =    56
  pure?   =    No
  max L   =     1

Basis: cc-pvdz-minao
  nao     =   168
  ncart   =   168
  nprim   =   728
  nshell  =   112
  natom   =    56
  pure?   =   Yes
  max L   =     1



In [4]:
####### =>  Build the pyGSM objects <= #########
# => level of theory <= #
printcool("Build the pyGSM level of theory (LOT) object")
mol=next(pb.readfile('pdb','data/system.pdb'))
coords = getAllCoords(mol)
atoms = getAtomicSymbols(mol)
geom= manage_xyz.combine_atom_xyz(atoms,coords)
states=[(1,0)]
lot = PyTC.from_options(states=states,psiw=psiw,geom=geom)

# => Create PES objects <= #
printcool("Building the PES objects")
pes = PES.from_options(lot=lot,ad_idx=0,multiplicity=1)


#========================================================#
#|     Build the pyGSM level of theory (LOT) object     |#
#========================================================#
 initializing LOT from geom
#========================================================#
#|               Building the PES objects               |#
#========================================================#


In [5]:
# => Molecule <= #
printcool("Build the pyGSM Molecule object \n with Translation and Rotation Internal Coordinates (TRIC)")
TRIC_molecule = Molecule.from_options(fnm='data/system.pdb',PES=pes,coordinate_type="TRIC")


#================================================================#
#|               Build the pyGSM Molecule object                |#
#|   with Translation and Rotation Internal Coordinates (TRIC)  |#
#================================================================#
 reading cartesian coordinates from file
 making primitives from options!
Build G: 33.800 Eig: 1.171
 making primitive Hessian
 forming Hessian in basis


In [6]:
printcool("Build the pyGSM Molecule object \n with Delocalized Coordinates (DLC)")
DLC_molecule = Molecule.from_options(fnm='data/system.pdb',PES=pes,coordinate_type="DLC")

#========================================================#
#|           Build the pyGSM Molecule object            |#
#|          with Delocalized Coordinates (DLC)          |#
#========================================================#
 reading cartesian coordinates from file
 building coordinate object
 making primitives from options!
Adding (20, 801) from minimum spanning tree
Adding (56, 61) from minimum spanning tree
Adding (58, 131) from minimum spanning tree
Adding (59, 78) from minimum spanning tree
Adding (60, 212) from minimum spanning tree
Adding (62, 235) from minimum spanning tree
Adding (63, 347) from minimum spanning tree
Adding (67, 128) from minimum spanning tree
Adding (68, 379) from minimum spanning tree
Adding (69, 146) from minimum spanning tree
Adding (72, 350) from minimum spanning tree
Adding (73, 356) from minimum spanning tree
Adding (74, 349) from minimum spanning tree
Adding (76, 338) from minimum spanning tree
Adding (82, 1448) from minimum spanning tree
Add

In [7]:
import timeit  
n=5

def update_coords(M):
    M.update_coordinate_basis()


In [9]:
printcool("Forming coordinate experiment!\n running update coords for nsteps={}".format(n))
total_time= timeit.timeit(stmt='update_coords(TRIC_molecule)',setup="from __main__ import update_coords, TRIC_molecule",number=n)
avg_time = total_time/n
print "total time = ",total_time
print "avg time = ",avg_time

#========================================================#
#|            Forming coordinate experiment!            |#
#|          running update coords for nsteps=5          |#
#========================================================#
Build G: 27.426 Eig: 1.257
Build G: 30.319 Eig: 0.468
Build G: 29.113 Eig: 0.237
Build G: 29.422 Eig: 0.220
Build G: 28.935 Eig: 0.238
total time =  147.665581942
avg time =  29.5331163883


In [10]:
printcool("Forming coordinate experiment!\n running update coords for nsteps={}".format(n))
total_time= timeit.timeit(stmt='update_coords(DLC_molecule)',setup="from __main__ import update_coords, DLC_molecule",number=n)
avg_time = total_time/n
print "total time = ",total_time
print "avg time = ",avg_time

#========================================================#
#|            Forming coordinate experiment!            |#
#|          running update coords for nsteps=5          |#
#========================================================#
Build G: 54.754 Eig: 345.752
Build G: 58.253 Eig: 342.692
Build G: 57.514 Eig: 336.293
Build G: 58.465 Eig: 346.812
Build G: 57.509 Eig: 349.775
total time =  2008.52382302
avg time =  401.704764605
